In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/ML_Task"

In [3]:
%cd /content/gdrive/My Drive/ML_Task

/content/gdrive/My Drive/ML_Task


In [4]:
import os
import time
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')
# For array manipulation
import numpy as np
import pandas as pd
import pandas.util.testing as tm
# For visualization
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import cv2
import imageio as io
from pylab import *
from sklearn.manifold import TSNE
#For model performance
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.externals import joblib
#For model training
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import tensorflow as tf
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.models import Sequential
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Conv2D, Activation, MaxPooling2D, UpSampling2D
from tensorflow.keras.optimizers import Adam, Adagrad, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras.models import load_model
from keras.preprocessing import image
import imageio as io

In [5]:
def plot_(x,row,col,ind,title):
    plt.subplot(row,col,ind)
    plt.imshow(x)
    plt.title(title)
    plt.axis('off')

In [6]:
def results_(query,result):
    def read(label):
        img = io.imread(label)
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        return image
    plt.figure(figsize=(10,5))
    if type(query)!=type(30):
        plot_(query,1,1,1,"Query Image")
    else:
        plot_(read(files[query]),1,1,1,"Query Image "+files[query])
    plt.show()
    plt.figure(figsize=(20,5)) 
    for iter,i in enumerate(result):
        plot_(read(files[i]),1,len(result),iter+1,files[i])
    plt.show()

In [7]:
def predictions(label,N=10,isurl=False):
    start_time = time.time()
    if isurl:
        img = io.imread(label)
        img = cv2.resize(img,(224,224))
    else:
        img = label
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data,axis=0)
    img_data = preprocess_input(img_data)
    feature = K.function([model.layers[0].input],[model.layers[12].output])
    feature = np.array(feature).flatten().reshape(1,-1)
    res = knn.kneighbors(feature.reshape(1,-1),return_distance=True,n_neighbors=N)
    results_(img,list(res[1][0])[1:])
    print("Time taken : ",np.round(time.time()-start_time,2)," sec")

In [13]:
# files = np.load('/content/drive/My Drive/files.npy')
optimizer = Adam(learning_rate=0.0005)
knn = joblib.load('knn_model.pkl')
kmeans = joblib.load('kmeans_model.pkl')
model = load_model('encoder_model.h5') 
model.compile(optimizer=optimizer, loss='mse')

In [18]:
query_path = 'http://tak-apps-dev.s3.amazonaws.com/recruitment/machine-learning/dataset/DSC_0025_4.JPG'
predictions(query_path,10,isurl=True)